In [10]:
import json
import requests
from pprint import pprint

In [11]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-py-datasource"
skillset_name = "cogsrch-py-skillset"
index_name = "cogsrch-py-index"
indexer_name = "cogsrch-py-indexer"

In [12]:
# Setup the endpoint
endpoint = 'https://my-new-search-service.search.windows.net'
headers = {'Content-Type': 'application/json',
           'api-key': 'x1vDjhDRD70ZPkbQGoPW12S4RrrdfS1jSAAhbFzK7eAzSeA2xYjp'}
params = {
    'api-version': '2020-06-30'
}

In [15]:
# Create a data source
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=blobstoragewestus;AccountKey=dpc88suMCvJCQqfnMKHMnIcTXBr3Ol2J0Qpoq8tf7OcKg15QK3Gkldgqg5Ip5syKFW5o922+HH0g+AStqSI2pg==;EndpointSuffix=core.windows.net"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "blobstoragewestus"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

204


In [37]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description":
    "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "categories": ["Organization"],
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "languageCode",
                    "targetName": "languageCode"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "textSplitMode": "pages",
            "maximumPageLength": 4000,
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                },
                {
                    "name": "languageCode",
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode", 
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        }
    ]
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

201


In [38]:
# Create an index
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false",
            "sortable": "true"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "sortable": "false",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "languageCode",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "keyPhrases",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "organizations",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [22]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "fieldMappings": [
        {
            "sourceFieldName": "metadata_storage_path",
            "targetFieldName": "id",
            "mappingFunction":
            {"name": "base64Encode"}
        },
        {
            "sourceFieldName": "content",
            "targetFieldName": "content"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/organizations",
            "targetFieldName": "organizations"
        },
        {
            "sourceFieldName": "/document/pages/*/keyPhrases/*",
            "targetFieldName": "keyPhrases"
        },
        {
            "sourceFieldName": "/document/languageCode",
            "targetFieldName": "languageCode"
        }
    ],
    "parameters":
    {
        "maxFailedItems": 5,
        "maxFailedItemsPerBatch": 5,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}
print(indexer_payload)
print(headers)
print(params)
r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r)

{'name': 'cogsrch-py-indexer', 'dataSourceName': 'cogsrch-py-datasource', 'targetIndexName': 'cogsrch-py-index', 'skillsetName': 'cogsrch-py-skillset', 'fieldMappings': [{'sourceFieldName': 'metadata_storage_path', 'targetFieldName': 'id', 'mappingFunction': {'name': 'base64Encode'}}, {'sourceFieldName': 'content', 'targetFieldName': 'content'}], 'outputFieldMappings': [{'sourceFieldName': '/document/organizations', 'targetFieldName': 'organizations'}, {'sourceFieldName': '/document/pages/*/keyPhrases/*', 'targetFieldName': 'keyPhrases'}, {'sourceFieldName': '/document/languageCode', 'targetFieldName': 'languageCode'}], 'parameters': {'maxFailedItems': 5, 'maxFailedItemsPerBatch': 5, 'configuration': {'dataToExtract': 'contentAndMetadata', 'imageAction': 'generateNormalizedImages'}}}
{'Content-Type': 'application/json', 'api-key': 'x1vDjhDRD70ZPkbQGoPW12S4RrrdfS1jSAAhbFzK7eAzSeA2xYjp'}
{'api-version': '2020-06-30'}
<Response [400]>
